In [22]:
#coding:utf-8
import pandas as pd
import time

from sklearn.metrics import log_loss
def time2cov(time_):
    '''
    时间是根据天数推移，所以日期为脱敏，但是时间本身不脱敏
    :param time_: 
    :return: 
    '''
    return time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time_))
def pre_process(data):
    '''
    :param data: 
    :return: 
    '''

    print('预处理')
    print('item_category_list_ing')
    for i in range(3):
        data['category_%d'%(i)] = data['item_category_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_category_list']

    print('item_property_list_ing')
    for i in range(3):
        data['property_%d'%(i)] = data['item_property_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_property_list']

    print('context_timestamp_ing')
    data['context_timestamp'] = data['context_timestamp'].apply(time2cov)

    print('time')
    data['context_timestamp_tmp'] = pd.to_datetime(data['context_timestamp'])
    data['week'] = data['context_timestamp_tmp'].dt.weekday
    data['hour'] = data['context_timestamp_tmp'].dt.hour
    data['minute'] = data['context_timestamp_tmp'].dt.minute
    data['halfhour'] = (data['hour']+1)*3+(data['minute']/20).astype('int')
    del data['context_timestamp_tmp']
    del data['minute']
    del data['week']
    del data['hour']
    del data['predict_category_property']
    # del data['predict_property_1']
    # del data['predict_property_2']
#     del data['shop_score_service']
#     del data['shop_score_description']
#     del data['shop_review_positive_rate']
#     del data['shop_score_delivery']
    return data

In [43]:
print('train')
train = pd.read_csv('train.csv',index_col=False)
dataa = pd.read_csv('train_all.csv',index_col=False)

train


## 特征选择

In [44]:
dataa.columns

Index(['instance_id', 'hour_num', 'item_click', 'shop_score_service_num',
       'shop_score_description_num', 'shop_review_positive_rate_num',
       'shop_score_delivery_num', 'user_click', 'property_0_click',
       'property_1_click', 'property_2_click', 'shop_click', 'brand_click',
       'category_1_pvnum', 'property_0_pvnum', 'property_1_pvnum',
       'property_2_pvnum', 'item_id_pvnum', 'occupation_click',
       'user_gender_click'],
      dtype='object')

In [45]:
tezheng = ['instance_id','item_id_pvnum', 'occupation_click','category_1_pvnum']

In [46]:
dataa = dataa[tezheng]

In [47]:

train = pd.merge(train,dataa,on=['instance_id'],how='left')
train = pre_process(train)

预处理
item_category_list_ing
item_property_list_ing
context_timestamp_ing
time


In [48]:
all_data = train.copy()


print('all_shape',train.shape)
print(train['context_timestamp'].max())
val = train[train['context_timestamp']>'2018-09-22 23:59:59']

# 空去 22 号

train = train[train['context_timestamp']<='2018-09-22 23:59:59']

# train = train[train['context_timestamp']>'2018-09-19 23:59:59']
print(train.shape)
print(val.shape)


all_shape (478087, 34)
2018-09-24 23:59:47
(357066, 34)
(121021, 34)


In [49]:

print('test')
test_a = pd.read_csv('test1.csv',index_col=False)
data2a = pd.read_csv('test1_all.csv',index_col=False) 

# del train[['user_id', 'category_1', 'property_2', 'property_0']]
# del val[['user_id', 'category_1', 'property_2', 'property_0']]
# del test_a[['user_id', 'category_1', 'property_2', 'property_0']]
# 这里是增加的内容


test


## 特征选择

In [50]:
data2a = data2a[tezheng]

In [51]:
test_a.columns

Index(['instance_id', 'item_id', 'item_category_list', 'item_property_list',
       'item_brand_id', 'item_city_id', 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level', 'user_id', 'user_gender_id',
       'user_age_level', 'user_occupation_id', 'user_star_level', 'context_id',
       'context_timestamp', 'context_page_id', 'predict_category_property',
       'shop_id', 'shop_review_num_level', 'shop_review_positive_rate',
       'shop_star_level', 'shop_score_service', 'shop_score_delivery',
       'shop_score_description'],
      dtype='object')

In [52]:
test_a = pd.merge(test_a,data2a,on=['instance_id'],how='left')
print(test_a.shape)
test_a = pre_process(test_a)
del train['item_city_id']
del val['item_city_id']
del test_a['item_city_id']
del train['shop_id']
del val['shop_id']
del test_a['shop_id']
# del train['context_id']
# del val['context_id']
# del test_a['context_id']
# del train['item_brand_id']
# del val['item_brand_id']
# del test_a['item_brand_id']
# del train['category_0']
# del val['category_0']
# del test_a['category_0']
# del train['category_2']
# del val['category_2']
# del test_a['category_2']
# del train['property_1']
# del val['property_1']
# del test_a['property_1']
# del train['property_2']
# del val['property_2']
# del test_a['property_2']

(42888, 29)
预处理
item_category_list_ing
item_property_list_ing
context_timestamp_ing
time


In [53]:

import datetime
def get_count_feat(all_data,data,long=3):
    end_time = data['context_timestamp'].min()
    begin_time = pd.to_datetime(end_time) - datetime.timedelta(days=long)
    all_data['context_timestamp'] = pd.to_datetime(all_data['context_timestamp'])
    all_data = all_data[
        (all_data['context_timestamp']<end_time)&(all_data['context_timestamp']>=begin_time)
                    ]
    
    print(end_time)
    print(begin_time)
    print(all_data['context_timestamp'].max()-all_data['context_timestamp'].min())
    item_count = all_data.groupby(['item_id'],as_index=False).size().reset_index()
    item_count.rename(columns={0:'item_count'},inplace=True)

    user_count = all_data.groupby(['user_id'], as_index=False).size().reset_index()
    user_count.rename(columns={0: 'user_count'}, inplace=True)
    return user_count,item_count




In [54]:

train_user_count,train_item_count = get_count_feat(all_data,train,2)

test_user_count,test_item_count = get_count_feat(all_data,test_a,2)

val_user_count,val_item_count = get_count_feat(all_data,val,2)
# print(train.columns)
train = pd.merge(train,train_user_count,on=['user_id'],how='left')
train = pd.merge(train,train_item_count,on=['item_id'],how='left')
train = train.fillna(-1)
val = pd.merge(val,val_user_count,on=['user_id'],how='left')
val = pd.merge(val,val_item_count,on=['item_id'],how='left')
val = val.fillna(-1)
test_a = pd.merge(test_a,test_user_count,on=['user_id'],how='left')
test_a = pd.merge(test_a,test_item_count,on=['item_id'],how='left')
test_a = test_a.fillna(-1)

# 删掉上面增加的用户点击量与商品点击量
# del train['user_count']
# del train['item_count']
# del val['user_count']
# del val['item_count']
# del test_a['user_count']
# del test_a['item_count']

# user_item_count = pd.read_csv


y_train = train.pop('is_trade')
train_index = train.pop('instance_id')

y_val = val.pop('is_trade')
val_index = val.pop('instance_id')
test_index = test_a.pop('instance_id')

print(test_a.shape)
del train['context_timestamp']
del val['context_timestamp']
del test_a['context_timestamp']
del all_data


2018-09-18 00:00:01
2018-09-16 00:00:01
NaT
2018-09-25 00:00:11
2018-09-23 00:00:11
1 days 23:59:33
2018-09-23 00:00:03
2018-09-21 00:00:03
1 days 23:59:43
(42888, 32)


In [55]:

print('baseline ing ... ...')
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from scipy import sparse
from sklearn.linear_model import LogisticRegression
# print(test_a.columns)

enc = OneHotEncoder()
lb = LabelEncoder()
# feat_set = list(test_a.columns)

del train['user_id']
del test_a['user_id']
del val['user_id']
# del train['item_id']
# del test_a['item_id']
# del val['item_id']
# 8288

print(train.columns)

# 自己使用onehot编码
train['x_train'] = 1
test_a['x_test'] = 1
val['x_val'] = 1

all_data = pd.concat([train,test_a,val])
all_data = pd.get_dummies(all_data)
print(all_data.shape)

X_train = all_data.loc[all_data['x_train'].notnull()]
X_test = all_data.loc[all_data['x_test'].notnull()]
X_val = all_data.loc[all_data['x_val'].notnull()]

del X_train['x_train']
del X_train['x_test']
del X_train['x_val']
del X_test['x_train']
del X_test['x_test']
del X_test['x_val']
del X_val['x_train']
del X_val['x_test']
del X_val['x_val']


baseline ing ... ...
Index(['item_id', 'item_brand_id', 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level', 'user_gender_id',
       'user_age_level', 'user_occupation_id', 'user_star_level', 'context_id',
       'context_page_id', 'shop_review_num_level', 'shop_review_positive_rate',
       'shop_star_level', 'shop_score_service', 'shop_score_delivery',
       'shop_score_description', 'item_id_pvnum', 'occupation_click',
       'category_1_pvnum', 'category_0', 'category_1', 'category_2',
       'property_0', 'property_1', 'property_2', 'halfhour', 'user_count',
       'item_count'],
      dtype='object')
(520975, 589)


In [56]:
# LogisticRegression?

In [57]:

# lr = LogisticRegression(C=5.0,penalty='l1')

# lr.fit(X_train, y_train)


In [58]:

# proba_val = lr.predict_proba(X_val)[:,1]
# proba_sub = lr.predict_proba(X_test)[:,1]
# print(log_loss(y_train,lr.predict_proba(X_train)[:,1]))
# print(log_loss(y_val,proba_val))
# print()

In [59]:

import lightgbm as lgb
gbm = lgb.LGBMRegressor(objective='binary',
                        num_leaves=50,
#                         max_depth = 8,
                        
                        learning_rate=0.01,
                        n_estimators=2000,
                        colsample_bytree = 0.65,
                        subsample = 0.7,
                        reg_alpha = 2
                       )
gbm.fit(X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='binary_logloss',
        early_stopping_rounds=150)



[1]	valid_0's binary_logloss: 0.683897
Training until validation scores don't improve for 150 rounds.
[2]	valid_0's binary_logloss: 0.674838
[3]	valid_0's binary_logloss: 0.665946
[4]	valid_0's binary_logloss: 0.657232
[5]	valid_0's binary_logloss: 0.648686
[6]	valid_0's binary_logloss: 0.640306
[7]	valid_0's binary_logloss: 0.632079
[8]	valid_0's binary_logloss: 0.624014
[9]	valid_0's binary_logloss: 0.616093
[10]	valid_0's binary_logloss: 0.608318
[11]	valid_0's binary_logloss: 0.600686
[12]	valid_0's binary_logloss: 0.593198
[13]	valid_0's binary_logloss: 0.585847
[14]	valid_0's binary_logloss: 0.578627
[15]	valid_0's binary_logloss: 0.571531
[16]	valid_0's binary_logloss: 0.56456
[17]	valid_0's binary_logloss: 0.557708
[18]	valid_0's binary_logloss: 0.550977
[19]	valid_0's binary_logloss: 0.544369
[20]	valid_0's binary_logloss: 0.537871
[21]	valid_0's binary_logloss: 0.531486
[22]	valid_0's binary_logloss: 0.5252
[23]	valid_0's binary_logloss: 0.519025
[24]	valid_0's binary_logloss

[203]	valid_0's binary_logloss: 0.125783
[204]	valid_0's binary_logloss: 0.125234
[205]	valid_0's binary_logloss: 0.124689
[206]	valid_0's binary_logloss: 0.124153
[207]	valid_0's binary_logloss: 0.123623
[208]	valid_0's binary_logloss: 0.123101
[209]	valid_0's binary_logloss: 0.122586
[210]	valid_0's binary_logloss: 0.122074
[211]	valid_0's binary_logloss: 0.121568
[212]	valid_0's binary_logloss: 0.121072
[213]	valid_0's binary_logloss: 0.120579
[214]	valid_0's binary_logloss: 0.120092
[215]	valid_0's binary_logloss: 0.11961
[216]	valid_0's binary_logloss: 0.119136
[217]	valid_0's binary_logloss: 0.118669
[218]	valid_0's binary_logloss: 0.118211
[219]	valid_0's binary_logloss: 0.117753
[220]	valid_0's binary_logloss: 0.117302
[221]	valid_0's binary_logloss: 0.116857
[222]	valid_0's binary_logloss: 0.116417
[223]	valid_0's binary_logloss: 0.115983
[224]	valid_0's binary_logloss: 0.115556
[225]	valid_0's binary_logloss: 0.115131
[226]	valid_0's binary_logloss: 0.114712
[227]	valid_0's b

[402]	valid_0's binary_logloss: 0.0856765
[403]	valid_0's binary_logloss: 0.0856362
[404]	valid_0's binary_logloss: 0.0855971
[405]	valid_0's binary_logloss: 0.0855579
[406]	valid_0's binary_logloss: 0.0855186
[407]	valid_0's binary_logloss: 0.0854781
[408]	valid_0's binary_logloss: 0.0854407
[409]	valid_0's binary_logloss: 0.0854023
[410]	valid_0's binary_logloss: 0.0853612
[411]	valid_0's binary_logloss: 0.0853232
[412]	valid_0's binary_logloss: 0.0852882
[413]	valid_0's binary_logloss: 0.0852519
[414]	valid_0's binary_logloss: 0.0852176
[415]	valid_0's binary_logloss: 0.0851807
[416]	valid_0's binary_logloss: 0.0851438
[417]	valid_0's binary_logloss: 0.0851097
[418]	valid_0's binary_logloss: 0.0850737
[419]	valid_0's binary_logloss: 0.085041
[420]	valid_0's binary_logloss: 0.0850079
[421]	valid_0's binary_logloss: 0.084976
[422]	valid_0's binary_logloss: 0.0849453
[423]	valid_0's binary_logloss: 0.0849116
[424]	valid_0's binary_logloss: 0.0848793
[425]	valid_0's binary_logloss: 0.08

[600]	valid_0's binary_logloss: 0.0825709
[601]	valid_0's binary_logloss: 0.0825678
[602]	valid_0's binary_logloss: 0.082564
[603]	valid_0's binary_logloss: 0.082561
[604]	valid_0's binary_logloss: 0.0825584
[605]	valid_0's binary_logloss: 0.0825568
[606]	valid_0's binary_logloss: 0.0825517
[607]	valid_0's binary_logloss: 0.0825469
[608]	valid_0's binary_logloss: 0.0825401
[609]	valid_0's binary_logloss: 0.0825355
[610]	valid_0's binary_logloss: 0.0825326
[611]	valid_0's binary_logloss: 0.0825275
[612]	valid_0's binary_logloss: 0.082524
[613]	valid_0's binary_logloss: 0.0825202
[614]	valid_0's binary_logloss: 0.082516
[615]	valid_0's binary_logloss: 0.0825131
[616]	valid_0's binary_logloss: 0.0825087
[617]	valid_0's binary_logloss: 0.0825058
[618]	valid_0's binary_logloss: 0.0825005
[619]	valid_0's binary_logloss: 0.0824972
[620]	valid_0's binary_logloss: 0.082495
[621]	valid_0's binary_logloss: 0.0824903
[622]	valid_0's binary_logloss: 0.0824872
[623]	valid_0's binary_logloss: 0.08248

[799]	valid_0's binary_logloss: 0.0821545
[800]	valid_0's binary_logloss: 0.082155
[801]	valid_0's binary_logloss: 0.0821535
[802]	valid_0's binary_logloss: 0.0821524
[803]	valid_0's binary_logloss: 0.082151
[804]	valid_0's binary_logloss: 0.0821511
[805]	valid_0's binary_logloss: 0.0821489
[806]	valid_0's binary_logloss: 0.0821465
[807]	valid_0's binary_logloss: 0.0821469
[808]	valid_0's binary_logloss: 0.0821458
[809]	valid_0's binary_logloss: 0.0821454
[810]	valid_0's binary_logloss: 0.0821456
[811]	valid_0's binary_logloss: 0.0821438
[812]	valid_0's binary_logloss: 0.0821438
[813]	valid_0's binary_logloss: 0.0821432
[814]	valid_0's binary_logloss: 0.0821448
[815]	valid_0's binary_logloss: 0.0821431
[816]	valid_0's binary_logloss: 0.0821418
[817]	valid_0's binary_logloss: 0.082141
[818]	valid_0's binary_logloss: 0.082141
[819]	valid_0's binary_logloss: 0.082141
[820]	valid_0's binary_logloss: 0.0821405
[821]	valid_0's binary_logloss: 0.0821388
[822]	valid_0's binary_logloss: 0.08213

[1000]	valid_0's binary_logloss: 0.0820583
[1001]	valid_0's binary_logloss: 0.0820582
[1002]	valid_0's binary_logloss: 0.0820597
[1003]	valid_0's binary_logloss: 0.0820605
[1004]	valid_0's binary_logloss: 0.0820608
[1005]	valid_0's binary_logloss: 0.0820603
[1006]	valid_0's binary_logloss: 0.0820586
[1007]	valid_0's binary_logloss: 0.0820586
[1008]	valid_0's binary_logloss: 0.0820599
[1009]	valid_0's binary_logloss: 0.0820594
[1010]	valid_0's binary_logloss: 0.0820577
[1011]	valid_0's binary_logloss: 0.0820581
[1012]	valid_0's binary_logloss: 0.0820581
[1013]	valid_0's binary_logloss: 0.0820577
[1014]	valid_0's binary_logloss: 0.0820577
[1015]	valid_0's binary_logloss: 0.0820579
[1016]	valid_0's binary_logloss: 0.0820557
[1017]	valid_0's binary_logloss: 0.0820561
[1018]	valid_0's binary_logloss: 0.0820565
[1019]	valid_0's binary_logloss: 0.0820574
[1020]	valid_0's binary_logloss: 0.0820566
[1021]	valid_0's binary_logloss: 0.0820576
[1022]	valid_0's binary_logloss: 0.0820567
[1023]	vali

[1195]	valid_0's binary_logloss: 0.0820542
[1196]	valid_0's binary_logloss: 0.0820548
[1197]	valid_0's binary_logloss: 0.0820546
[1198]	valid_0's binary_logloss: 0.0820559
[1199]	valid_0's binary_logloss: 0.0820564
[1200]	valid_0's binary_logloss: 0.0820555
[1201]	valid_0's binary_logloss: 0.0820552
[1202]	valid_0's binary_logloss: 0.0820557
[1203]	valid_0's binary_logloss: 0.0820571
[1204]	valid_0's binary_logloss: 0.0820565
[1205]	valid_0's binary_logloss: 0.0820575
[1206]	valid_0's binary_logloss: 0.0820555
[1207]	valid_0's binary_logloss: 0.0820572
[1208]	valid_0's binary_logloss: 0.0820573
[1209]	valid_0's binary_logloss: 0.0820558
[1210]	valid_0's binary_logloss: 0.0820547
[1211]	valid_0's binary_logloss: 0.0820527
[1212]	valid_0's binary_logloss: 0.0820531
[1213]	valid_0's binary_logloss: 0.0820534
[1214]	valid_0's binary_logloss: 0.082053
[1215]	valid_0's binary_logloss: 0.0820528
[1216]	valid_0's binary_logloss: 0.0820546
[1217]	valid_0's binary_logloss: 0.0820547
[1218]	valid

[1391]	valid_0's binary_logloss: 0.0820635
[1392]	valid_0's binary_logloss: 0.0820633
[1393]	valid_0's binary_logloss: 0.0820642
[1394]	valid_0's binary_logloss: 0.0820636
[1395]	valid_0's binary_logloss: 0.0820647
[1396]	valid_0's binary_logloss: 0.0820653
[1397]	valid_0's binary_logloss: 0.0820649
[1398]	valid_0's binary_logloss: 0.0820658
[1399]	valid_0's binary_logloss: 0.0820649
[1400]	valid_0's binary_logloss: 0.0820646
[1401]	valid_0's binary_logloss: 0.0820652
[1402]	valid_0's binary_logloss: 0.0820639
[1403]	valid_0's binary_logloss: 0.082064
[1404]	valid_0's binary_logloss: 0.082065
[1405]	valid_0's binary_logloss: 0.0820645
[1406]	valid_0's binary_logloss: 0.0820648
[1407]	valid_0's binary_logloss: 0.0820645
[1408]	valid_0's binary_logloss: 0.0820641
[1409]	valid_0's binary_logloss: 0.082065
[1410]	valid_0's binary_logloss: 0.082065
[1411]	valid_0's binary_logloss: 0.0820646
[1412]	valid_0's binary_logloss: 0.082064
[1413]	valid_0's binary_logloss: 0.0820639
[1414]	valid_0's

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.65,
       learning_rate=0.01, max_depth=-1, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=2000,
       n_jobs=-1, num_leaves=50, objective='binary', random_state=None,
       reg_alpha=2, reg_lambda=0.0, silent=True, subsample=0.7,
       subsample_for_bin=200000, subsample_freq=1)

In [60]:
print('Start predicting...')
# predict
y_pred_1 = gbm.predict(X_val, num_iteration=gbm.best_iteration_ )
y_sub_1 = gbm.predict(X_test, num_iteration=gbm.best_iteration_ )
print(log_loss(y_train,gbm.predict(X_train)))
print(log_loss(y_val,y_pred_1))

# print(log_loss(y_val,(y_pred_1 + proba_val)/2))#0.0816995451384

Start predicting...
0.0810279477581
0.0820459569246


In [63]:

# import numpy as np
# bes = 1
# for i in np.arange(0,1,0.01):
#     if(log_loss(y_val,(i * y_pred_1 + (1-i) * proba_val)) < bes):
#         bes = log_loss(y_val,(y_pred_1 + proba_val)/2)
# print(bes)#0.083131236937

# xx_analyse = pd.DataFrame()
# xx_analyse['ture'] = list(y_val)
# xx_analyse['pre'] = list(proba_val)
# xx_analyse['pre_1'] = list(y_pred_1)
# xx_analyse.to_csv('temp.csv',index=False)
#
# sub = pd.DataFrame()
# sub['instance_id'] = list(test_index)
# sub['predicted_score'] = list(y_sub_1)
# sub.to_csv('feather/20180327.txt',sep=" ",index=False)

In [64]:
y_sub_1.shape

(42888,)

In [65]:
sub = pd.DataFrame()
sub['instance_id'] = list(test_index)
sub['predicted_score'] = list(y_sub_1)
sub.to_csv('20180417.txt',sep=" ",index=False)